# Exploring LangChain 

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## <center>LLM Chain</center>

We initialized the model

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

We can begin asking questions right away using `invoke` method.  
The message contains several outputs but with `content` attribute we can see the response.  
This question isn't in the training data so it shouldn't be accurate

In [4]:
llm.invoke('''
Decime que paso hoy en la camara de diputados de Argentina y que se estaba votando
''').content

'Hoy en la Cámara de Diputados de Argentina se estaba votando el proyecto de ley de reforma del Ministerio Público Fiscal, que busca modificar el funcionamiento de este organismo encargado de investigar delitos en el país. La votación ha generado un fuerte debate entre los diferentes bloques políticos y se espera que sea una sesión muy intensa.'

We can also guide the response with a prompt template  
we use the `from_messages` method to pass a list of tuples containing the system message for the _**AI**_ and the future input from the _**user**_

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en politica Argentina y un maestro en redacciones concisas"),
    ("user", "{input}")
])

Combine both the `llm` and the `prompt` into a simple **chain** using the [LCEL](https://python.langchain.com/docs/expression_language/) syntax

In [6]:
chain = prompt | llm

We can now invoke it and passed question to the input placeholder of the user tuple.  
It still won't know the answer, but should respond according to the system prompt  

In [7]:
chain.invoke({"input": "Decime que paso hoy en la camara de diputados de Argentina y que se estaba votando"})

AIMessage(content='Hoy en la Cámara de Diputados de Argentina se estaba votando el proyecto de ley de reforma del Impuesto a las Ganancias, que busca aumentar el mínimo no imponible y reducir la carga impositiva para los trabajadores. Finalmente, la ley fue aprobada con amplio consenso y contó con el respaldo de la mayoría de los bloques políticos presentes en la sesión.', response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 50, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-69b98771-b956-4936-b67e-06f04e8a7992-0')

Since this output being from a ChatModel is a `message` object is more convenient to work with strings. For that we add a simple output parser

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

We create the chain again, adding the parser

In [9]:
chain = prompt | llm | output_parser

We call it again, this time the output being a string

In [10]:
chain.invoke(
    {"input": 
        '''
        Decime que paso hoy en la camara de diputados de Argentina y que se estaba votando. 
        Tambien menciona al final la fecha de hoy
        '''}
)

'Hoy en la Cámara de Diputados de Argentina se votó el proyecto de ley de reforma del impuesto a las Ganancias, que busca aumentar el piso a partir del cual se comienza a pagar el tributo. La fecha de hoy es [fecha actual].'

## <center>Retrieval Chain</center>

Para realmente responder la pregunta cuya respuesta no esta siendo la correcta necesitamos proveer contexto adicional to the LLM. We can do this via _retrieval_  

through this process we will look up relevant documents from a _retriever_.  
A Retriever can be backed by anything, a SQL table, the internet, Videos, PDF's files, etc. In this instance we will populate a [vector database](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.lanacion.com.ar/politica/gritos-y-tension-en-diputados-pedidos-de-ansioliticos-e-insultos-en-el-recinto-durante-el-debate-por-nid29042024/")

docs = loader.load()

Ahora hay que indexarlo in the vector store usando algun embeddings model

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Build index

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# instanciate the splitter
text_splitter = RecursiveCharacterTextSplitter()

# split the documents
documents = text_splitter.split_documents(docs)

# create the embeddings and store them
vector = FAISS.from_documents(documents, embeddings)

Ahora que está todo indexado creamos el _retrieval chain_. Va a agarrar la pregunta, mirar los documentos relevantes, pasar esos documentos juntos con la pregunta al LLM y hacer la pregunta original.  
En este caso usamos el método `from_template` de `ChatPromptTemplate`

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only in the context provided:

    <context>
    {context}
    <context>

    Question: {input}
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)


### Side note

Podriamos pasar directamente los documentos si quisieramos y lo responderia correctamente (of corso)

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Que paso hoy en la camara de diputados?",
    "context": [Document(page_content="Se vota como tratar la ley de bases")]
})

'Hoy en la Cámara de Diputados se votó sobre cómo tratar la ley de bases.'

Sin embargo, la idea es que los documentos vengan del retriever que hicimos antes, de esa manera se puede el retriever para seleccionar dinámicamente los documentos más relevantes para la pregunta en cuestión

In [16]:
from langchain.chains import create_retrieval_chain

# creamos el retriever
retriever = vector.as_retriever()
# creamos la retriever chain con los documentos, pregunta y retriever
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Pum ahora la podemos invocar, esto nos da un `dict`, la respuesta está en la key  `answer`

In [17]:
response = retrieval_chain.invoke({
    "input": "Que paso hoy en la camara de diputados y que tema se estaba tratando/votando"
})

print(response["answer"])

Hoy en la Cámara de Diputados hubo fuertes intercambios y tensiones durante el debate por la Ley de Bases y el paquete fiscal impulsado por el Gobierno de Javier Milei. Se escucharon gritos, acusaciones e insultos entre los legisladores, incluyendo un pedido de ansiolíticos. El debate se centró en el ordenamiento de la discusión en el recinto, que finalmente se definió sería en particular por capítulos y de manera nominal.


## GOING BEYOND

## <center> Conversational Retrieval Chain</center>

Hasta ahora nuestra **RC** solo responde de a una pregunta. Hoy son populares los chatbots los cuales podés interactuar de manera _conversacional_  

Dicho esto hay que cambiar sobre que input trabaja el retrieval method, hay que hacer que tome en cuenta el _historial_ de conversacion entero en vez de solo el input mas reciente, de igual forma el LLM final que responde  

Lo que tenemos que hacer es crear una nueva _chain_ que tome el input mas reciente **y** el historial de conversacion y usar un LLM para generar una _search query_

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# prompt para el LLM asi genera el serach query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Podemos testearlo pasando una pregunta subsecuente contra un determinado history

In [19]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Que paso hoy en la camara de diputados?"),
                AIMessage(content='''
                
                Hoy en la Cámara de Diputados hubo tensiones
                y cruces entre legisladores durante la sesión
                para debatir la Ley de Bases y el paquete fiscal impulsado
                por el gobierno de Javier Milei.
                Los diputados estaban discutiendo
                y votando sobre la mencionada ley y el paquete fiscal.

                ''')]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Entre quienes hubo tension?"
})


[Document(page_content='Los cruces previosPrevio a lograr el quorum con 135 diputados, la sesión comenzó con tensiones debido a los cruces y acusaciones entre los legisladores de la Cámara de Diputados. El primer conflicto se dio segundos después de que Menem abriera la jornada debido a desacuerdos sobre cómo funcionaría el ordenamiento de la discusión en el recinto, que finalmente se definió sería en particular por capítulos y de manera nominal. “Arrancamos mal”, lamentó Germán Martínez -jefe de la bancada de UP-, y agregó: “En labor parlamentaria fijamos una posición sobre que los temas merecen un tratamiento diferencial. No alcanza con decir que el paquete fiscal estuvo incluido en el expediente; el volumen que alcanzó la cuestión fiscal -junto con el debate de la ley ómnibus- requieren un tratamiento diferenciado”.El diputado Germán Martínez, durante la sesión en el Congreso.Fabián MarelliAdemás, cuestionó que unificar temas representa “un límite en la libertad de expresión de los 

Esto nos da documentos acerca de que paso hoy en la cámara de diputados, esto se debe a que el LLM genero una nueva query combinando el *chat_history* con la pregunta subsecuente

Ahora que tenés este **retriever crea querys basado en contexto y preguntas subsecuentets**, podes crear una nueva _chain_ para seguir la conversación con estos artículos en mente

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [21]:
chat_history = [HumanMessage(content="Que paso hoy en la camara de diputados?"),
                AIMessage(content='''
                
                Hoy en la Cámara de Diputados hubo tensiones
                y cruces entre legisladores durante la sesión
                para debatir la Ley de Bases y el paquete fiscal impulsado
                por el gobierno de Javier Milei.
                Los diputados estaban discutiendo
                y votando sobre la mencionada ley y el paquete fiscal.

                ''')]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Entre quienes hubo tension?"
})
response

{'chat_history': [HumanMessage(content='Que paso hoy en la camara de diputados?'),
  AIMessage(content='\n                \n                Hoy en la Cámara de Diputados hubo tensiones\n                y cruces entre legisladores durante la sesión\n                para debatir la Ley de Bases y el paquete fiscal impulsado\n                por el gobierno de Javier Milei.\n                Los diputados estaban discutiendo\n                y votando sobre la mencionada ley y el paquete fiscal.\n\n                ')],
 'input': 'Entre quienes hubo tension?',
 'context': [Document(page_content='Los cruces previosPrevio a lograr el quorum con 135 diputados, la sesión comenzó con tensiones debido a los cruces y acusaciones entre los legisladores de la Cámara de Diputados. El primer conflicto se dio segundos después de que Menem abriera la jornada debido a desacuerdos sobre cómo funcionaría el ordenamiento de la discusión en el recinto, que finalmente se definió sería en particular por capítu

Si disecamos la respuesta entre la selva veremos que la respuesta es coherente dado el contexto

In [22]:
print(response["answer"])

Entre los legisladores de la Cámara de Diputados, hubo tensiones y acusaciones durante el debate por la Ley de Bases y el paquete fiscal. Algunos de los involucrados en los cruces fueron el diputado Germán Martínez, la diputada Silvana Giudici, la diputada Myriam Bregman, la diputada Carolina Gaillard, y la diputada Karina Banfi.


## <center>Agents</centter>

Our current chains knows each step ahead of time. Now is time for _Agents_ to decide what actions to take based on the tools we give them  

we will give them access to two tools:

1. The retriever created before
2. A search tool to answer question requiring up-to-date information


In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "Incidente_diputados_search",
    "Busca informacion acerca del incidente en la camara de diputados. Para cualquier pregunta acerca de diputados, You must use this tool!" 
)

For the search tool we'll use [Tavily](https://python.langchain.com/docs/integrations/retrievers/tavily/) 

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

Now we create a list of tools for the _Agent_

In [25]:
tools = [retriever_tool, search]

With the tools ready we just need to create the Agent

In [26]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

# Get the prompt to use (se puede cambiar)
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Ahora podemos invocar al agente y preguntarle del incidente

In [27]:
agent_response = agent_executor.invoke({
    "input": "Dame solo los nombres de las personas involucradas en el incidente en diputados Argentina de hoy"
})

print(agent_response["output"])



> Entering new AgentExecutor chain...

Invoking: `Incidente_diputados_search` with `{'query': 'incidente en diputados Argentina hoy'}`


CERRARCERRARSECCIONESInicioSeccionesMis NotasClub LNPerfilBuscá en LA NACION...Últimas noticiasTránsito y transporteClimaLA NACION DataPolíticaEconomíaDólar HoyCampoPropiedadesComercio ExteriorMovilidadÍndicesEl MundoEstados UnidosSociedadBuenos AiresSeguridadEducaciónCulturaComunidadBienestarCienciaHablemos de todoOpiniónEditorialesColumnistasCartas de SuscriptoresDeportesFútbolRugbyTenisCanchallenaLifestyleLN JuegosTurismoTecnologíaHoróscopoFeriadosLoterías y quinielasRecetasPodcastsModa y BellezaEspectáculosCartelera de cineCartelera de teatroEdición ImpresaAcceso PDFEditorialesConversaciones de domingoSábadoIdeasCarta de lectoresAvisos fúnebresAvisos socialesRevistasRevista OHLALÁ!Revista ¡HOLA!Revista LugaresRevista LivingRevista Rolling StoneRevista JardínLN+Exclusivo SuscriptoresKiosco LA NACIONClub LA NACION>LA NACION>PolíticaGritos y tensió

In [28]:
agent_response = agent_executor.invoke({
    "input": "Como esta el clima en Buenos Aires ahora mismo?. Dame la temperatura en grados celcius"
})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'clima en Buenos Aires'}`


[{'url': 'https://www.meteored.com.ar/tiempo-en_Buenos+Aires-America+Sur-Argentina-Ciudad+Autonoma+de+Buenos+Aires-SABE-1-13584.html', 'content': 'Tiempo en Buenos Aires - Pronóstico del tiempo a 14 días. Los datos sobre el Tiempo, temperatura, velocidad del viento, la humedad, la cota de nieve, presión, etc . Buenos Aires Argentina 18. Mar del Plata Provincia de Buenos Aires 16. La Plata Provincia de Buenos Aires 17. Bahía Blanca Provincia de Buenos Aires 17. Tiempo; Noticias;'}, {'url': 'https://www.tiempo.com/buenos-aires.htm', 'content': 'Consulta el pronóstico del tiempo para Buenos Aires hoy y los próximos días. Temperatura, lluvia, viento, nubosidad y más datos meteorológicos actualizados.'}, {'url': 'https://www.accuweather.com/en/ar/buenos-aires/7894/weather-forecast/7894', 'content': 'Get the current and future weather conditions for Buenos Aires, Argent

We can have conversation

In [29]:
chat_history = [HumanMessage(content="Habia alguien de apellido Menem en la camara de diputados?"), 
               AIMessage(content="Si, se llama Martin Menem")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Cual es su cargo / funcion en la camara de diputados?"
})



> Entering new AgentExecutor chain...

Invoking: `Incidente_diputados_search` with `{'query': 'Martin Menem'}`


Los cruces previosPrevio a lograr el quorum con 135 diputados, la sesión comenzó con tensiones debido a los cruces y acusaciones entre los legisladores de la Cámara de Diputados. El primer conflicto se dio segundos después de que Menem abriera la jornada debido a desacuerdos sobre cómo funcionaría el ordenamiento de la discusión en el recinto, que finalmente se definió sería en particular por capítulos y de manera nominal. “Arrancamos mal”, lamentó Germán Martínez -jefe de la bancada de UP-, y agregó: “En labor parlamentaria fijamos una posición sobre que los temas merecen un tratamiento diferencial. No alcanza con decir que el paquete fiscal estuvo incluido en el expediente; el volumen que alcanzó la cuestión fiscal -junto con el debate de la ley ómnibus- requieren un tratamiento diferenciado”.El diputado Germán Martínez, durante la sesión en el Congreso.Fabián MarelliAde

{'chat_history': [HumanMessage(content='Habia alguien de apellido Menem en la camara de diputados?'),
  AIMessage(content='Si, se llama Martin Menem')],
 'input': 'Cual es su cargo / funcion en la camara de diputados?',
 'output': 'Martin Menem es el presidente de la Cámara de Diputados.'}

## <center>Serving with LangServe</center>

To create a server we need to make a `serve.py` file containing our logic for serving the application. 3 things  

1. The definition of chain
2. Our FastAPI App
3. A definition of a _route_ from which to serve the chain, done with `langserve.add_routes`